In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import urllib3
from datetime import datetime
import datetime as dt
import os
import time
import timeit

In [2]:
print(os.getcwd())
today=dt.datetime.today()
print(today)

C:\Users\cmartin\Desktop\Estadistica Andorra
2022-03-09 15:54:37.567670


In [3]:
lista_tablas= pd.read_excel("URLs_estadistica_ad.xlsx",sheet_name="Hoja1")
num_tablas= lista_tablas["Num tabla"].to_list()

In [4]:
lista_tablas.tail()

,Categoria,Subcategoria,Detalle,Frecuencia,Num tabla,Fecha inicio,Fecha final,Subtema,codi tema,Unitat,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,subtemes=,LEN,Hallar subtema=,Diferencia,LEN de subtema,Subtema.1
178,Societat i Condicions de vida,EPF base 2010,% HABITATGES PER TIPUS,Anual,2757,2010-01-01,2021-05-01,222,15,porcentaje,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,Transport i Telecomunicacions,ENTRADA DE VEHICLES PER LES FRONTERES,ENTRADA DE VEHICLES TOTAL,Mensual,448,2001-05-01,2021-05-01,69,11,vehicles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,Transport i Telecomunicacions,ENTRADA DE VEHICLES PER LES FRONTERES,ENTRADA DE VEHICLES PER TIPUS I FRONTERA,Mensual,451,2001-05-01,2021-05-01,69,11,vehicles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,Mercat laboral,SALARI MITJÀ (CASS I DE),SALARI MITJÀ PER ACTIVITAT (NIVELL 1) (CASS I DE),Mensual,846,1990-01-01,NaT,123,14,euros,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,Mercat laboral,SALARI MITJÀ (CASS I DE),SALARI MITJÀ PER ACTIVITAT I EDAT (CASS I DE),Mensual,847,1990-01-01,NaT,123,14,euros,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
lista_tablas['Frecuencia'].value_counts()

Anual         130
Mensual        42
Trimestral     11
Name: Frecuencia, dtype: int64

In [6]:
x = len(lista_tablas["Num tabla"])
print(x)

183


In [7]:
from os import path

def write_csv_df(path, filename, df):
    # Give the filename you wish to save the file to
    pathfile = os.path.normpath(os.path.join(path,filename))

    # Use this function to search for any files which match your filename
    files_present = os.path.isfile(pathfile) 
    # if no matching files, write to csv, if there are matching files, print statement
    if not files_present:
        df.to_csv(pathfile, sep=';')
    else:
        df.to_csv(pathfile, sep=';',mode="a", header=False)
        

In [8]:
print('INICIANDO DESCARGA')
print('----------------------------')


lista_resp_http=[]
#for i in range(100,110):
for i in range(0,x):  
    
    try:
        # 1 - leemos la fecha y la convertimos para que la pueda incorporar en la URL
        año = str(lista_tablas.iloc[i,5].year)  #seleccionamos solo el año
        mes = str(lista_tablas.iloc[i,5].month)
        dia = str(lista_tablas.iloc[i,5].day)
        if len(dia) ==1:
            day = "0"+dia
        else:
            day = dia
            #print(day)    
        if len(mes) ==1:
            month = "0"+mes 
        else:
            month = mes
            #print(day)
        fecha_inicio = day+"/"+month+"/"+año
        # 2- seleccionamos el numero de tabla para obtener la informacion deseada
        num_tabla=lista_tablas.iloc[i,4]
        #3- seleccionamos el subtema, que permite acceder a la seccion que contiene la informacion
        subtema = lista_tablas.iloc[i,7]
        # 4- codigo tema
        codi_tema= lista_tablas.iloc[i,8]
        print(fecha_inicio)
        pd.set_option("display.max_rows",None)
        
        # 5 - Categoria,subcategoria,frecuencia
        categoria = lista_tablas.iloc[i,0]
        subcategoria = lista_tablas.iloc[i,1]
        frecuencia = lista_tablas.iloc[i,3]
        detalle = lista_tablas.iloc[i,2]
        unidades = lista_tablas.iloc[i,9]
        num_tabla=lista_tablas.iloc[i,4]
        subtema = lista_tablas.iloc[i,7]
        codi_tema= lista_tablas.iloc[i,8]
           
        url="https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1="+fecha_inicio+"&any2=01/03/2022&codi_divisio="+str(num_tabla)+"&lang=1&codi_subtemes="+str(subtema)+"&codi_tema="+str(codi_tema)
        subtema = lista_tablas.iloc[i,7]
        http= urllib3.PoolManager()
        r = http.request('GET', url)
        print("Num de iteracion: %d - num tabla:  %d respuesta http : %d" %(i,num_tabla,r.status))
        response = r.data
        str_data= response.decode('latin-1')
        lista_resp_http.append(r.status)
        #print(lista_resp_http)

        str_data=str_data.replace('\r\n\t\t\t\t\t\t\t--','0;').replace('\r\n\t\t\t\t\t\t','0;')
        
        lines=str_data.splitlines() #dividimos el str_Data en lineas
        columns=lines[0].split(";") # el numero de columnas sale de dividir el primer elemento por su separador ";"
        columns.pop() # eliminamos el ultimo elemento de cada fila porque esta vacio
        num_col=len(columns) #contamos el num de elemntos de la primera fila

        n=0
        lista_dif=[]
        while n< len(lines):

            if len(lines[n].split(";")) == num_col:
                diferencia= len(lines[n].split(";"))-num_col-1
                #print("el numero de elementos en {} es {}  " .format(n,(len(lines[n].split(";")))))
                lista_dif.append(diferencia)
            else:
                diferencia= len(lines[n].split(";"))-num_col-1
                #print("la diferencia en el elemento %d es de %d" % (n,diferencia))
                lista_dif.append(diferencia)

            n+=1

        counter = 0
        main_dict = {}
        for col in columns:
            main_dict[col] = []

        counter=0
        for line in lines:
            #for counter in range(1,len(lines):
            if counter>0:
                values = line.strip().split(";")
                delete_values_last_element=values.pop()
                if lista_dif[counter]==0:
                    for i in range(len(columns)):
                        main_dict[columns[i]].append(values[i])
                else:
                    for i in range(len(columns)):
                        if i==0:
                            main_dict[columns[i]].append(values[i])
                        else:
                            main_dict[columns[i]].append(values[lista_dif[counter]+i]) #REVISAR AQUI!!!!!!
            counter +=1


        print("El data set tiene %d filas y %d columnas"%(counter-1, num_col)) 
        print('----------------------------')
        print("La tabla obtenida proviene de:" + str(url))
        print('----------------------------')

        df_imports=pd.DataFrame(main_dict)

        df_imports_index=df_imports.iloc[:,0]
        df_values=df_imports.iloc[:,1:]
        columns_of_values=df_values.columns
        type(columns_of_values)
        columns_of_values

        num_fechas=len(columns_of_values)

        for elem in columns_of_values:
            df_values[elem]=df_values[elem].str.replace(".","").str.replace(",",".").str.replace("%","").astype(float)

        pd.options.display.float_format = '{:,.2f}'.format # con esta opcion cambiamos el formato exponencial hacia decimal con separacion de miles
        df_values.describe()

        new_df_imports=pd.merge(df_imports_index,df_values,left_index=True, right_index=True)


        n,j=new_df_imports.shape
        lista_final=[]
        for i in range(1,j):

            for k in range(0,n):

                periodo=new_df_imports.columns[i]
                parametro=new_df_imports.iloc[k,0]
                valor=new_df_imports.iloc[k,i]
                lista=[periodo,parametro,valor]
                lista_final.append(lista)

        df_final=pd.DataFrame(data=lista_final, columns=['Fecha','Parametro','Valor'])

        df_final['Fecha'] = pd.to_datetime(df_final['Fecha'])

        df_final.set_index("Fecha",inplace=True)
        
        # 1- PARROQUIAS
        
        df_parroquias =[]

        for p in df_final['Parametro']:
            if 'Andorra' in p:
                df_parroquias.append("Andorra")
            elif "Encamp" in p:
                df_parroquias.append("Encamp")
            elif "Escaldes" in p:
                df_parroquias.append("Escaldes")
            elif "Ordino" in p:
                df_parroquias.append("Ordino")
            elif "Massana" in p:
                df_parroquias.append("La Massana")
            elif "Julià" in p:
                df_parroquias.append("Sant Julià")
            elif "Canillo" in p:
                df_parroquias.append("Canillo")
            elif "Pas" in p:
                df_parroquias.append("Pas de la Casa")
            else:
                df_parroquias.append(" ")
        df_parroquias

        df_final['Parroquia'] = df_parroquias
        
        # 2- tipo de alojamiento
        
        df_tipo_aloj =[]

        for aloj in df_final['Parametro']:
            if 'Hotel' in aloj:
                df_tipo_aloj.append("Hotel")
            elif "Aparthotel" in aloj:
                df_tipo_aloj.append("Aparthotel")
            elif "Apartament" in aloj:
                df_tipo_aloj.append("Apartament turistic")
            elif "Camping" in aloj:
                df_tipo_aloj.append("Camping")
            elif ("amics" or "familiar") in aloj:
                df_tipo_aloj.append("Vivenda d'amics o familiars")
            elif ("Vivenda pr" or "lloguer") in aloj:
                df_tipo_aloj.append("Vivenda propia o de lloguer")
            elif "Hostal" in aloj:
                df_tipo_aloj.append("Hostal o residencia")
            elif "Pensio" in aloj:
                df_tipo_aloj.append("Pensio")
            elif "Altres" in aloj:
                df_tipo_aloj.append( " ")
            elif "1 estrella" in aloj:
                df_tipo_aloj.append("1 estrella")
            elif "2 estrelles" in aloj:
                df_tipo_aloj.append("2 estrelles")
            elif "3 estrelles" in aloj:
                df_tipo_aloj.append("3 estrelles")
            elif "4 estrelles" in aloj:
                df_tipo_aloj.append("4 estrelles")
            elif "5 estrelles" in aloj:
                df_tipo_aloj.append("5 estrelles")
            else:
                df_tipo_aloj.append(" ")
            
        df_tipo_aloj

        df_final['Tipo de alojamiento'] = df_tipo_aloj
        
        # 3- tipo de transporte
        
        df_transporte =[]

        for trans in df_final['Parametro']:
            if 'Cotxe' in trans:
                df_transporte.append("Cotxe")
            elif "Carabana" in trans:
                df_transporte.append("Carabana")
            elif "Autobus" in trans:
                df_transporte.append("Autobus")
            elif "Moto" in trans:
                df_transporte.append("Moto")
            elif "Camionetes" in trans:
                df_transporte.append("Camionetes")
            elif "Camions" in trans:
                df_transporte.append("Camions")
            elif "Motocicleta" in trans:
                df_transporte.append("Motocicleta")
            elif "Ciclomotors" in trans:
                df_transporte.append("Ciclomotors")
            elif "Turismes" in trans:
                df_transporte.append("Turismes")
            elif "Furgonetes" in trans:
                df_transporte.append("Furgonetes")
            elif "Vehicles de transport de persones" in trans:
                df_transporte.append("Vehicles de transport de persones")
            elif "Semi remolc" in trans:
                df_transporte.append("Semi remolc")
            elif "Remolc industrial" in trans:
                df_transporte.append("Remolc industrial")
            elif "Remolcs particulars" in trans:
                df_transporte.append("Remolcs particulars")
            elif "Tractors agricoles" in trans:
                df_transporte.append("Tractors agricoles")
            elif "Vehicles obres pÃºbliques" in trans:
                df_transporte.append("Vehicles obres pÃºbliques")
            else:
                df_transporte.append(" ")
        
            
        df_transporte

        df_final['Tipo de vehiculo'] = df_transporte
        
        # 4- Sexo
        
        df_genero = []
        
        for sex in df_final['Parametro']:
            if 'Home' in sex:
                df_genero.append('Home')
            elif 'Dona' in sex:
                df_genero.append('Dona')
            else:
                df_genero.append(' ')
        
        df_final['Genero']=df_genero
        
       
        # 5 - Tipo de inmueble
        
        df_inmuebles=[]
        
        for inm in df_final['Parametro']:
            if 'Pis, habitatge unifamiliar i edifici' in inm:
                df_inmuebles.append(' Pis, habitatge unifamiliar i edifici')
            elif "Terreny i plaça d'aparcament" in inm:
                df_inmuebles.append("Terreny i plaça d'aparcament")
            elif "Local comercial i magatzem (nau)" in inm:
                df_inmuebles.append("Local comercial i magatzem (nau)")
            elif 'Pis' in inm:
                df_inmuebles.append('Pis')
            elif 'Habitatge unifamiliar' in inm:
                df_inmuebles.append('Habitatge unifamiliar')
            elif 'habitatge' in inm:
                df_inmuebles.append('Habitatge')
            elif 'Terreny' in inm:
                df_inmuebles.append('Terreny')
            elif 'Aparcament' in inm:
                df_inmuebles.append('Aparcament')
            elif 'aparcament' in inm:
                df_inmuebles.append('Aparcament')
            elif 'Local Comercial' in inm:
                df_inmuebles.append('Local Comercial')
            elif 'local comercial' in inm:
                df_inmuebles.append('Local Comercial')
            elif 'Magatzem' in inm:
                df_inmuebles.append('Magatzem')
            elif 'xalet' in inm:
                df_inmuebles.append('Xalet')
            elif 'magatzem' in inm:
                df_inmuebles.append('Magatzem')
            elif 'Edifici' in inm:
                df_inmuebles.append('Edifici')
            elif 'Altres construccions' in inm:
                df_inmuebles.append('Altres construccions')    
            elif 'Nombre de permisos de xalet' in inm:
                df_inmuebles.append('Xalet') 
            elif "Nombre de permisos d'habitatge" in inm:
                df_inmuebles.append('Habitatges') 
            elif "Nombre de permisos de magatzem" in inm:
                df_inmuebles.append('Magatzem') 
            elif "Nombre de permisos de despatx" in inm:
                df_inmuebles.append('Despatx')
            elif "despatx" in inm:
                df_inmuebles.append('Despatx')
            elif "Nombre de permisos de hotel" in inm:
                df_inmuebles.append('Hotel')
            elif "Plaça aparcament" in inm:
                df_inmuebles.append('Parking')
            elif "aparcament en habitatges" in inm:
                df_inmuebles.append('Parking')
            elif "Local comercial" in inm:
                df_inmuebles.append('Local comercial')
            else:
                df_inmuebles.append(' ')
        
        df_final['Tipo de inmueble']=df_inmuebles
                       
        # 6 - Tipo energia
        
        
        df_energy=[]
        
        for en in df_final['Parametro']:
            if "Gasoil" in en:
                df_energy.append("Gasoil")
            elif "Gasolines" in en:
                df_energy.append("Gasolines")
            elif "Elèctric 100%" in en:
                df_energy.append("Elèctric 100%")
            elif "Híbrid gasolina endollable" in en:
                df_energy.append("Híbrid gasolina endollable")
            elif "Híbrid gasolina no endollable" in en:
                df_energy.append("Híbrid gasolina no endollable")
            elif "Híbrid gasoil endollable" in en:
                df_energy.append("Híbrid gasoil endollable")
            elif "Híbrid gasoil no endollable" in en:
                df_energy.append("Híbrid gasoil no endollable")
            elif "Sense combustible o no disponible" in en:
                df_energy.append("Sense combustible o no disponible")
            elif "Electricitat" in en:
                df_energy.append("Electricitat")
            elif "Gasolina" in en:
                df_energy.append("Gasolina")
            elif "Gasoil locomoció" in en:
                df_energy.append("Gasoil locomocio")
            elif "Fuel domèstic" in en:
                df_energy.append("Fuel domestic")
            elif "Butà" in en:
                df_energy.append("Buta")
            elif "Propà" in en:
                df_energy.append("Propa")
            elif "Carbó" in en:
                df_energy.append("Carbo")   
            else:
                df_energy.append(" ")
                
        df_final['Tipo energia']=df_energy
        
        # 7- Consumo Energia
        
        df_EnergyConsump=[]
        
        for ene in df_final['Parametro']:
            if "Usos domèstics" in ene:
                df_EnergyConsump.append("Us domestic")
            elif "Construcció i annexes" in ene:
                df_EnergyConsump.append("Construccions i annexes")
            elif "Indústries" in ene:
                df_EnergyConsump.append("Industries")
            elif "Distribució" in ene:
                df_EnergyConsump.append("Distribucio")
            elif "Hoteleria i restauració" in ene:
                df_EnergyConsump.append("Hoteleria i restauracio")
            elif "Serveis financers" in ene:
                df_EnergyConsump.append("Serveis financers")
            elif "Altres serveis" in ene:
                df_EnergyConsump.append("Altres serveis")
            elif "Usos Enllumenat públic" in ene:
                df_EnergyConsump.append("Usos Enllumenat public")
            elif "Estacions esquí" in ene:
                df_EnergyConsump.append("Estacions esqui")
            elif "Administracions" in ene:
                df_EnergyConsump.append("Administracions")
            elif "Altres distribuïdores" in ene:
                df_EnergyConsump.append("Altres distribuidores")
            elif "Global" in ene:
                df_EnergyConsump.append("Global")
            else:
                df_EnergyConsump.append(" ")
                
        df_final['Usos energia'] = df_EnergyConsump
        
        # 8 edad
        
        df_rangos_edad=[]
        
        for edad in df_final['Parametro']:
            if "De 00-04" in edad:
                df_rangos_edad.append("De 00-04")
            elif "De 05-09" in edad:
                df_rangos_edad.append("De 05-09")
            elif "De 10-14" in edad:
                df_rangos_edad.append("De 10-14")
            elif "De 15-19" in edad:
                df_rangos_edad.append("De 15-19")
            elif "De 20-24" in edad:
                df_rangos_edad.append("De 20-24")
            elif "De 25-29" in edad:
                df_rangos_edad.append("De 25-29")
            elif "De 30-34" in edad:
                df_rangos_edad.append("De 30-34")
            elif "De 35-39" in edad:
                df_rangos_edad.append("De 35-39")
            elif "De 40-44" in edad:
                df_rangos_edad.append("De 40-44")
            elif "De 45-49" in edad:
                df_rangos_edad.append("De 45-49")
            elif "De 50-54" in edad:
                df_rangos_edad.append("De 50-54")
            elif "De 55-59" in edad:
                df_rangos_edad.append("De 55-59")
            elif "De 60-64" in edad:
                df_rangos_edad.append("De 60-64")
            elif "De 65-69" in edad:
                df_rangos_edad.append("De 65-69")
            elif "De 70-74" in edad:
                df_rangos_edad.append("De 70-74")
            elif "De 75-79" in edad:
                df_rangos_edad.append("De 75-79")
            elif "De 80-84" in edad:
                df_rangos_edad.append("De 80-84")
            elif "De més de 85" in edad:
                df_rangos_edad.append("De mas de 85")
            else:
                df_rangos_edad.append(" ")
                
        df_final['Rango edad']=df_rangos_edad
            
        
        # vamos a incorporar una columna llamada tematica
        filas,columnas=df_final.shape
        tema=np.repeat(categoria,filas)
        subcat = np.repeat(subcategoria,filas)
        frec = np.repeat(frecuencia,filas)
        det = np.repeat(detalle,filas)
        unit = np.repeat(unidades,filas)
        tabla=np.repeat(num_tabla,filas)
        subtema_codi=np.repeat(subtema,filas)
        codi_tema_codi=np.repeat(codi_tema,filas)
        
        
        df_final['Categoria']=tema
        df_final['Subcategoria']=subcat
        df_final['Frecuencia']=frec
        df_final['Detalle']=det
        df_final['Unidades']=unit
        df_final['Num_tabla']=tabla
        df_final['Num_subtema']=subtema_codi
        df_final['Num_codi_tema']=codi_tema_codi
       
        #C:\\Users\\cmart\\Desktop\\Scrapers
        #path = "/Users/cmart/Desktop/Scrapers"
        
        path="/Users/cmartin/Desktop/Estadistica Andorra"
        filename = "estadistica_historico_v3.csv"
        #filename = "estadistica_historico_"+str(today)+".csv"
        write_csv_df(path,filename,df_final)
        time.sleep(3)
    except:
        pass

  
print("descarga finalizada")



INICIANDO DESCARGA
----------------------------
01/01/2010
Num de iteracion: 0 - num tabla:  2162 respuesta http : 200
El data set tiene 9 filas y 13 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/01/2010&any2=01/03/2022&codi_divisio=2162&lang=1&codi_subtemes=8&codi_tema=2
----------------------------


<ipython-input-8-b07a69d33c06>:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_values[elem]=df_values[elem].str.replace(".","").str.replace(",",".").str.replace("%","").astype(float)


01/01/2010
Num de iteracion: 1 - num tabla:  2777 respuesta http : 200
El data set tiene 127 filas y 13 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/01/2010&any2=01/03/2022&codi_divisio=2777&lang=1&codi_subtemes=8&codi_tema=2
----------------------------
01/01/2010
Num de iteracion: 2 - num tabla:  2778 respuesta http : 200
El data set tiene 702 filas y 0 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/01/2010&any2=01/03/2022&codi_divisio=2778&lang=1&codi_subtemes=8&codi_tema=2
----------------------------
01/01/2010
Num de iteracion: 3 - num tabla:  2794 respuesta http : 200
El data set tiene 1402 filas y 0 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules

01/01/1970
Num de iteracion: 23 - num tabla:  156 respuesta http : 200
El data set tiene 8 filas y 53 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/01/1970&any2=01/03/2022&codi_divisio=156&lang=1&codi_subtemes=21&codi_tema=5
----------------------------
01/01/1970
Num de iteracion: 24 - num tabla:  157 respuesta http : 200
El data set tiene 19 filas y 51 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/01/1970&any2=01/03/2022&codi_divisio=157&lang=1&codi_subtemes=21&codi_tema=5
----------------------------
01/01/1970
Num de iteracion: 25 - num tabla:  160 respuesta http : 200
El data set tiene 8 filas y 53 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=ini

Num de iteracion: 45 - num tabla:  264 respuesta http : 200
El data set tiene 8 filas y 40 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/01/1982&any2=01/03/2022&codi_divisio=264&lang=1&codi_subtemes=45&codi_tema=8
----------------------------
01/01/1982
Num de iteracion: 46 - num tabla:  268 respuesta http : 200
El data set tiene 8 filas y 40 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/01/1982&any2=01/03/2022&codi_divisio=268&lang=1&codi_subtemes=45&codi_tema=8
----------------------------
01/01/1981
Num de iteracion: 47 - num tabla:  270 respuesta http : 200
El data set tiene 8 filas y 41 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/0

...

01/03/2010
Num de iteracion: 90 - num tabla:  2619 respuesta http : 200
El data set tiene 8 filas y 46 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/03/2010&any2=01/03/2022&codi_divisio=2619&lang=1&codi_subtemes=280&codi_tema=8
----------------------------
01/01/2010
Num de iteracion: 91 - num tabla:  2654 respuesta http : 200
El data set tiene 14 filas y 11 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/01/2010&any2=01/03/2022&codi_divisio=2654&lang=1&codi_subtemes=280&codi_tema=8
----------------------------
01/01/2010
Num de iteracion: 92 - num tabla:  2655 respuesta http : 200
El data set tiene 56 filas y 13 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?form

01/05/2009
Num de iteracion: 112 - num tabla:  894 respuesta http : 200
El data set tiene 6 filas y 34 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/05/2009&any2=01/03/2022&codi_divisio=894&lang=1&codi_subtemes=127&codi_tema=10
----------------------------
01/05/2009
Num de iteracion: 113 - num tabla:  898 respuesta http : 200
El data set tiene 6 filas y 35 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/05/2009&any2=01/03/2022&codi_divisio=898&lang=1&codi_subtemes=127&codi_tema=10
----------------------------
01/05/2009
Num de iteracion: 114 - num tabla:  888 respuesta http : 200
El data set tiene 8 filas y 33 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formul

01/03/2009
Num de iteracion: 134 - num tabla:  1210 respuesta http : 200
El data set tiene 5 filas y 156 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/03/2009&any2=01/03/2022&codi_divisio=1210&lang=1&codi_subtemes=37&codi_tema=7
----------------------------
01/03/2009
Num de iteracion: 135 - num tabla:  1211 respuesta http : 200
El data set tiene 35 filas y 156 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/03/2009&any2=01/03/2022&codi_divisio=1211&lang=1&codi_subtemes=37&codi_tema=7
----------------------------
01/01/2002
Num de iteracion: 136 - num tabla:  1101 respuesta http : 200
El data set tiene 1 filas y 9 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?for

01/01/2013
Num de iteracion: 156 - num tabla:  1970 respuesta http : 200
El data set tiene 1 filas y 9 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/01/2013&any2=01/03/2022&codi_divisio=1970&lang=1&codi_subtemes=222&codi_tema=15
----------------------------
01/01/2002
Num de iteracion: 157 - num tabla:  1113 respuesta http : 200
El data set tiene 11 filas y 9 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/01/2002&any2=01/03/2022&codi_divisio=1113&lang=1&codi_subtemes=143&codi_tema=15
----------------------------
01/01/2013
Num de iteracion: 158 - num tabla:  1972 respuesta http : 200
El data set tiene 13 filas y 9 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?fo

01/01/2010
Num de iteracion: 178 - num tabla:  2757 respuesta http : 200
El data set tiene 3 filas y 12 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/01/2010&any2=01/03/2022&codi_divisio=2757&lang=1&codi_subtemes=222&codi_tema=15
----------------------------
01/05/2001
Num de iteracion: 179 - num tabla:  448 respuesta http : 200
El data set tiene 1 filas y 250 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?formules=inici&any1=01/05/2001&any2=01/03/2022&codi_divisio=448&lang=1&codi_subtemes=69&codi_tema=11
----------------------------
01/05/2001
Num de iteracion: 180 - num tabla:  451 respuesta http : 200
El data set tiene 5 filas y 250 columnas
----------------------------
La tabla obtenida proviene de:https://www.estadistica.ad/serveiestudis/web/exportar_banc_dades_csv.asp?for

In [ ]:

print(" El tipo de respuesta http es : ")
print('\n')
lista_tablas['lista_resp_http']=lista_resp_http

print(" El num de tablas descargadas es : ")
print('\n')
lista_tablas['lista_resp_http'].value_counts()

In [ ]:
df_final.shape

In [ ]:
df_final

In [ ]:
df_final.info()

In [ ]:
path = "/Users/cmart/Desktop/Scrapers"
filename = "estadistica_historico_v3_13_2_22.csv"
write_csv_df(path,filename,df_final)

In [ ]:
today=dt.datetime.today()
today